# Imports

In [1]:
import pandas as pd
import geopandas as gpd
import googlemaps
from shapely.geometry import Point
import os
import numpy as np
import re
from textwrap import shorten
from bs4 import BeautifulSoup
import gspread

## Data read-in

In [2]:
df = pd.read_csv('Austin luxury map 12_9 - Sheet1.csv')

In [5]:
df

,Address,Price,Price psf
0,"700 Brandon Way Austin, Texas 78733",$5.25 million,$533 per square foot
1,"314 Reveille Rd West Lake Hills, Texas 78746",$2.79 million,$1060 per square foot
2,"1003 Kinney Ave Austin, Texas 78704",$2.499 million,"$1,189 per square foot"
3,"1414 Eva St Austin, Texas 78704",$2.4795 million,$900 per square foot
4,"17137 Majestic Ridge Rd Austin, Texas 78738",$2.4 million,$397 per square foot
5,"2704 Cherry Ln Austin, Texas 78703",$2.395 million,$516 per square foot
6,"2802 Deercreek Cir Austin, Texas 78703",$2 million,$447 per square foot


In [4]:
# Drop the 'URL' column if you no longer need it
df = df.drop(columns=['Photo (?)'])

In [4]:
# df['Description'].iloc[0]

## Geocode

In [6]:
df.columns

Index(['Address', 'Price', 'Price psf'], dtype='object')

In [12]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [14]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['Address'].apply(geocode)

In [17]:
# Convert 'geocoded' column into Shapely Point objects
df['geometry'] = df['geocoded'].apply(lambda coords: Point(coords[1], coords[0]))

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set the CRS to WGS84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

,Address,Price,Price psf,geocoded,geometry
0,"700 Brandon Way Austin, Texas 78733",$5.25 million,$533 per square foot,"(30.3237373, -97.85873900000001)",POINT (-97.85874 30.32374)
1,"314 Reveille Rd West Lake Hills, Texas 78746",$2.79 million,$1060 per square foot,"(30.2852831, -97.8049259)",POINT (-97.80493 30.28528)
2,"1003 Kinney Ave Austin, Texas 78704",$2.499 million,"$1,189 per square foot","(30.2573341, -97.764747)",POINT (-97.76475 30.25733)
3,"1414 Eva St Austin, Texas 78704",$2.4795 million,$900 per square foot,"(30.2494719, -97.7512465)",POINT (-97.75125 30.24947)
4,"17137 Majestic Ridge Rd Austin, Texas 78738",$2.4 million,$397 per square foot,"(30.3359912, -98.00191459999999)",POINT (-98.00191 30.33599)
5,"2704 Cherry Ln Austin, Texas 78703",$2.395 million,$516 per square foot,"(30.2936545, -97.7707532)",POINT (-97.77075 30.29365)
6,"2802 Deercreek Cir Austin, Texas 78703",$2 million,$447 per square foot,"(30.3072773, -97.767716)",POINT (-97.76772 30.30728)


In [18]:
gdf.columns

Index(['Address', 'Price', 'Price psf', 'geocoded', 'geometry'], dtype='object')

In [19]:
gdf = gdf.drop(columns={'geocoded':'Details'})

In [20]:
# Strip whitespace from all string columns
gdf.columns = gdf.columns.str.strip()  # Trim whitespace from column names
gdf = gdf.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Trim whitespace from string values

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_82704/2323350875.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  gdf = gdf.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Trim whitespace from string values


In [21]:
# Convert all data to strings if possible, or remove complex structures
for col in gdf.columns:
    if gdf[col].dtype == object:
        try:
            gdf[col] = gdf[col].astype(str)
        except Exception as e:
            print(f"Could not convert column {col}: {e}")
            gdf.drop(columns=[col], inplace=True)


In [22]:
gdf.to_file('data.geojson', driver='GeoJSON')

In [23]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Austin_Luxury_Map_12_9_24


In [24]:
gdf.columns

Index(['Address', 'Price', 'Price psf', 'geometry'], dtype='object')

<a href="https://therealdeal.com/chicago/2024/04/03/david-gassman-sells-chicago-apartment-portfolio-for-72m/"> 6954 North Sheridan Road, Chicago</a>